In [1]:
class A():
    def __new__(cls, *args, **kwargs):
        print('hi from __new__')
        return super().__new__(cls)

When an object of type type is called, `__new__` gets executed first.

In [2]:
A()

hi from __new__


If `__new__` returns an instance of type `cls`, it gets passed to `__init__`.

In [3]:
class B(A):
    def __init__(self):
        print('hi from __init__')

In [4]:
B()

hi from __new__
hi from __init__


If `__new__` returns an instace of type `!= cls`, `__init__` doesn't get called.

In [5]:
class A():
    def __new__(cls, *args, **kwargs):
        print('hi from __new__')
        return 42
    
class B(A):
    def __init__(self):
        print('hi from __init__')

In [6]:
B()

hi from __new__


42

A method defined on a class becomes an instance method on the object the class creates.

In [7]:
class Klass():
    def just_a_method_on_Klass(x): print(f'hi, I got a {x}')

In [8]:
Klass.just_a_method_on_Klass('sneaky suspicion')

hi, I got a sneaky suspicion


In [9]:
Klass().just_a_method_on_Klass() # got bound to the instance!

hi, I got a <__main__.Klass object at 0x7f78146c9910>


## Metaclasses

A metaclass can be defined.

In [10]:
class Meta(type): # need to inherit from type
    def __new__(cls, name, bases, dct): # same signature as type(...) apart from additional arg 0
        print('hi from __new__ @ meta')
        dct['signature'] = 'Leonardo di ser Piero da Vinci' # artist's signature
        klass = super().__new__(cls, name, bases, dct) # need to return an object of type cls 
                                                       # else __init__ won't get called
        print(klass)
        return klass
    
    def __init__(cls, name, bases, dct):
        print('hi from __init__ @ meta')

It creates and initializes an object of type type, a class.

In [11]:
class A(metaclass=Meta): pass

hi from __new__ @ meta
<class '__main__.A'>
hi from __init__ @ meta


In [12]:
A().signature

'Leonardo di ser Piero da Vinci'

A method defined on a metaclass behaves just like a method defined on a class.

In [13]:
class Meta(type):
    def just_a_method_on_Meta(x): print(f'hi, I got a {x}')

In [14]:
class A(metaclass=Meta): pass

In [15]:
Meta.just_a_method_on_Meta('doughnut')

hi, I got a doughnut


It becomes an instance method of the class a metaclass constructs.

In [16]:
A.just_a_method_on_Meta()

hi, I got a <class '__main__.A'>


In [17]:
A().just_a_method_on_Meta()

AttributeError: 'A' object has no attribute 'just_a_method_on_Meta'

When you put a method in a dct passed to the constructor (`__new__`) of type, it doesn't live on the metaclass.

It behaves like a method defined on a class.

In [18]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        dct['a_method_from_a_dict'] = lambda x: print(f'hi, I got a {x}')
        return  super().__new__(cls, name, bases, dct)

In [19]:
class A(metaclass=Meta): pass

In [20]:
Meta.a_method_from_a_dict()

AttributeError: type object 'Meta' has no attribute 'a_method_from_a_dict'

In [21]:
A.a_method_from_a_dict('cheesburger')

hi, I got a cheesburger


It becomes an instance method of an object that the class constructs.

In [22]:
A().a_method_from_a_dict()

hi, I got a <__main__.A object at 0x7f78146c9990>


These two are equivalent.

In [23]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        dct['p'] = lambda x: print(x)
        return super().__new__(cls, name, bases, dct)

class A_with_meta(metaclass=Meta): pass

class A_without_meta():
    def p(self): print(self)

In [24]:
A_with_meta.p('with meta')

with meta


In [25]:
A_without_meta.p('without meta')

without meta


In [26]:
A_with_meta().p()

In [27]:
A_without_meta()